# SQL
В решении задачи в качестве исходного дата-сета используйте приложенный CSV-файл dataset.csv.<br><br>
На входе у вас есть таблица orders, содержащая все заказы, совершенные в ОЗОН <br>
Описание таблицы orders:
<ul>
<li>user_id (string) - уникальный идентификатор пользователя</li>
<li>order_id (string) - уникальный идентификатор заказа</li>
<li>order_time (Int64) - время сделанного заказа в unixtime (секундны, UTC)</li>
<li>order_cost (float) - стоимость заказа</li>
<li>success_order_flg (bool) - идентификатор, определяющий был ли заказ успешно доставлен. Выполненный заказ - это заказ у которого в данной таблице success_order_flg=True</li>
</ul><br>

Пользователь считается привлеченным в день date, если он совершает первый заказ в день date.<br>
Пользователь считается реактивированным в день date, если он совершает заказ в день date, при этом не совершил ни одного заказа в течении предшествующих 90 дней, но совершал хотя бы 1 заказ за свою пользовательскую историю ранее.<br><br>
 
На выходе требуется получить таблицу:
<ul>
<li>date - календарная дата привлечения или реактивации</li>
<li>gmv360d_new - общая стоимость заказов за 360 дней после дня date, совершенных пользователями, привлеченными в день date</li>
<li>gmv360d_reactivated - общая стоимость заказов за 360 дней после дня date, совершенных пользователями, реактивированными в день date</li>
<li>users_count_new - кол-во пользователей, привлеченных в день date</li>
<li>users_count_reactivated- кол-во пользователей, реактивированных в день date</li>
</ul><br>
Заказы, совершенные пользователем после очередной реактивации, не должны учитываться в расчете GMV для предыдущих периодов активности пользователя. Т.е., например, если пользователь пришел в день 0, совершал заказы по день 30, не совершал заказов до дня 130 и совершил заказ в день 130, его заказы в дни 130-490 должны учитываться только в gmv360d_reactivated для дня 130, но не в gmv360d_new дня 0.<br><br>
То же правило распространяется на повторные реактивации пользователя. То есть если у рассмотренного нами пользователя на 230-й день жизни снова произошла реактивация, то gmv360d_reactivated будет посчитано для предыдущего периода за дни 130-229, а для итогового, с реактивацией на 230-й день, это поле будет рассчитано за дни 230-590. 
Приложите так же итоговый датасет в формате CSV.<br><br>

Тестовое задание можно выполнить в нотации удобной для вас БД.<br>
Нотацию просьба указать, чтобы ориентироваться на неё при проверке задания.

##### Примечание:
К сожалению, у меня нигде нет доступа к БД. Запрос был написан "вслепую" прямо здесь.<br>
Уверен, что тут есть ошибки - запрос требует отладки.<br>
Тем не менее, скелет выглядил бы как-то так...<br><br>
Нотация БД - Teradata.

In [ ]:
query = '''
--Финальная таблица
select
    gmv_rea.date_,
    gmv_new.gmv360d_new,
    gmv_rea.gmv360d_reactivated,
    sum(case when gmv_new.gmv360d_new not null then 1 else 0) as users_count_new,
    sum(case when gmv_new.gmv360d_reactivated not null then 1 else 0) as users_count_reactivated
from (
    select
        t1.user_id,
        act_per.date_,
        (act_per.date_number - act_per.active_periods_counter) as continuity_cntr_,
        sum(t1.order_cost) over(partition by t1.user_id, continuity_cntr_ order by act_per.date_) as gmv360d_reactivated
    from (
        --Периоды активности Юзера
        select
            a.user_id,
            cast(a.order_time as date) as date_,
            a.date_number,
            row_number() over(partition by a.user_id order by date_) as active_periods_counter,
            min() over(partition by a.user_id order by date_ rows between 1 preceding and 1 preceding) as last_active_date,
            max() over(partition by a.user_id order by date_ rows between 1 following and 1 following) as next_active_date
        from (
            --Дата привлечения Юзера + его Реактивация на дату
            select
                orders_cnt.user_id,
                orders_cnt.order_time,
                orders_cnt.date_number,
                case
                    when min(orders_cnt.order_time) over(partition by orders_cnt.user_id order by orders_cnt.order_time) = orders_cnt.order_time then 'True'
                    else 'False'
                end as new_user
                case
                    when (orders_cnt.orders_cnt_last90days = 0) and (orders_cnt.success_order_flg = 'True') and (min(orders_cnt.order_time) over(partition by orders_cnt.user_id order by orders_cnt.order_time) < orders_cnt.order_time) then 'True'
                    else 'False'
                end as reactivated,
                case
                    when new_user = 'True' or reactivated = 'True' then 'True'
                    else 'False'
                end as activity
            from (
                --Кол-во выполненных заказов юзера за последние 90 дней (не считая заказ текущей строки)
                select
                    t1.user_id,
                    t1.order_time,
                    rn_dates.date_number,
                    t1.success_order_flg
                    count(t2.order_id) - 1 as orders_cnt_last90days
                from orders t1
                left join (
                        select
                            user_id,
                            order_time,
                            order_id
                        from orders
                        where
                            success_order_flg = 'True'
                        )t2 on t1.user_id = t2.user_id
                               and t1.order_time between (t2.order_time - 90) and t2.order_time
                left join (
                        --Нумерация дат
                        select
                            date_,
                            row_number() over(order by dates_) as date_number
                        from (select distinct cast(order_time as date) as date_ from orders)dst_dates
                            )rn_dates on cast(t1.order_time as date) = rn_dates.date_
                group by 1, 2, 3, 4)orders_cnt
                )a
        where
            1=1
            and a.activity = 'True'
            )act_per
    left join (
            --Таблица заказов
            select
                *
            from orders
            )t1 on t1.user_id = act_per.user_id
                    and cast(t1.order_time as date) between act_per.last_active_date and act_per.next_active_date
    )gmv_rea
left join (
        --Сумма заказов юзера за следующие 360 дней
        select
            t1.user_id,
            t1.order_time,
            t1.success_order_flg
            sum(t2.order_id) as gmv360d_new
        from orders t1
        left join (
                select
                    user_id,
                    order_time,
                    order_id
                from orders
                where
                    success_order_flg = 'True'
                )t2 on t1.user_id = t2.user_id
                       and t1.order_time between t2.order_time and (t2.order_time + 359)
        group by 1, 2, 3
        qualify row_number() over(partition by t1.user_id order by t1.order_time) = 1
        )gmv_new
'''